# Pieces in Network file - Data cleaning after combining daily files


1.   Once the 6 months or any number of "Pieces in network scripts" files are combined into one dataset we use the below set of code blocks to clean the data
2.   There are few functions that help us aggreated the data setup by setup without loosing any valueable information







### Global Settings and Imports

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Cleaning Functions

In [ ]:
# Fucntion to load the "six_month_network_data.csv" or the "new_df.parquet" file both are same we use the .parquet file to load the files faster
def load_data():
    df = pd.read_parquet('/content/drive/MyDrive/MGT6748_Steelcase/new_df.parquet')
    return df

def filter_data(df, start_date, end_date, rdc_list):
    df = df.loc[(df['report_date'] >= start_date) & (df['report_date'] <= end_date)]
    df = df[df['Shipping Point'].isin(rdc_list)]
    return df

def get_barcode_first_date(df):
    barcode_first_date = df.groupby(['Barcode','Location'])['report_date'].min().reset_index()
    barcode_first_date.columns = ['Barcode','Location','first_date']
    return barcode_first_date

def merge_first_date(df, barcode_first_date):
    df = df.merge(barcode_first_date, on=['Barcode','Location'])
    return df

def get_barcode_last_date(df):
    barcode_last_date = df.groupby(['Barcode','Location'])['report_date'].max().reset_index()
    barcode_last_date.columns = ['Barcode','Location','last_date']
    return barcode_last_date

def merge_last_date(df, barcode_last_date):
    df = df.merge(barcode_last_date, on=['Barcode','Location'])
    return df

def get_barcode_total_inv(df):
    barcode_total_inv = df.groupby(['report_date','Location']).size().reset_index()
    barcode_total_inv.columns = ['report_date','Location','total_inv']
    return barcode_total_inv

def merge_total_inv(df, barcode_total_inv):
    df = df.merge(barcode_total_inv, on=['report_date','Location'])
    return df

def calculate_age(df):
    df['age'] = (df['report_date'] - df['first_date']).dt.days
    df['total_age'] = (df['last_date'] - df['first_date']).dt.days
    #df['total_vol'] = df.apply(lambda x: x['Volume'] if x['age'] == 0 else 0, axis=1)
    df['incoming_inv'] = df.apply(lambda x: 1 if x['age'] == 0 else 0, axis=1)
    df['outgoing_inv'] = df.apply(lambda x: 1 if x['age'] == x['total_age'] else 0, axis=1)
    return df

def categorize_age(age):
    if age == 0:
        return 'Age_0'
    elif age == 1:
        return 'Age_1'
    elif age == 2:
        return 'Age_2'
    elif age == 3:
        return 'Age_3'
    elif age == 4:
        return 'Age_4'
    elif age == 5:
        return 'Age_5'
    elif age == 6:
        return 'Age_6'
    elif age == 7:
        return 'Age_7'
    elif age > 7:
        return 'Age>7_days'
    else:
        return f'Age {age}'

def categorize_ages(df):
    df['age_category'] = df['age'].apply(categorize_age)
    return df

def calculate_clean_data(df, rdc, sqft, barcode_total_inv):
    df = df[['Barcode', 'Shipping Point', 'Location','report_date','first_date','last_date','age','total_age','incoming_inv','outgoing_inv','age_category','Volume']]
    clean_df = df.groupby(['report_date','Location','age_category']).agg(total=('report_date', 'count'), inc=('incoming_inv', 'sum'),out=('outgoing_inv', 'sum'),dwel_time=('age', 'mean'),total_vol=('Volume', 'sum'))
    clean_df.reset_index(drop=False, inplace=True)
    clean_df['rdc'] = rdc
    clean_df = merge_total_inv(clean_df, barcode_total_inv)
    clean_df["days_of_week"] = clean_df["report_date"].dt.dayofweek
    clean_df['report_week'] = (clean_df["report_date"] -pd.to_timedelta(clean_df["days_of_week"], unit='d'))
    clean_df['usable_sq_ft'] = sqft
    clean_df['usable_sq_ft'] = clean_df['usable_sq_ft'].astype(int)
    clean_df['vol_multi'] = 0.464
    clean_df['vol_multi'] = clean_df['vol_multi'].astype(float)
    clean_df['space_util'] = clean_df['total_vol']*clean_df['vol_multi']
    clean_df['space_util'] = clean_df['space_util'].astype(float)
    clean_df['inv_turnover_ratio'] = clean_df['out']/clean_df['total_inv']
    clean_df['inv_turnover_ratio'] = clean_df['inv_turnover_ratio'].astype(float)
    clean_df['days_of_inventory'] = 1/clean_df['inv_turnover_ratio'].replace({ 0 : np.nan })
    clean_df['days_of_inventory'] = round(clean_df['days_of_inventory'].astype(float),1)
    return clean_df

def save_data_to_csv(dataframe, file_path):
    # Check if file exists
    file_exists = os.path.isfile(file_path)

    # Append data to existing file or create new file and append data
    with open(file_path, 'a', newline='') as f:
        dataframe.to_csv(f, sep='|', quotechar='"', index=False, header=not file_exists)

def save_data_to_parquet(dataframe, file_path):
    # Check if file exists
    file_exists = os.path.isfile(file_path)

    # Append data to existing file or create new file and append data
    if file_exists:
        df = pd.read_parquet(file_path)
        df = pd.concat([df, dataframe], axis=0)
        df.reset_index(drop=True, inplace=True)
        df.to_parquet(file_path)
    else:
        dataframe.to_parquet(file_path, index=False)


In [ ]:
# function to clean up and aggregate the 18M records from the 6 months 'pieces in network script' file
def process_data(rdc_name, rdc_sqft):
    gm = load_data()
    gm1 = filter_data(gm, '2022-09-01', '2022-12-30', [rdc_name])
    gm2 = gm1.loc[gm1['Shipping Point'] == rdc_name]
    barcode_first_date = get_barcode_first_date(gm2)
    gm2 = merge_first_date(gm2, barcode_first_date)
    barcode_last_date = get_barcode_last_date(gm2)
    gm2 = merge_last_date(gm2, barcode_last_date)
    barcode_total_inv = get_barcode_total_inv(gm2)
    gm2 = calculate_age(gm2)
    gm2 = categorize_ages(gm2)
    clean_gm = calculate_clean_data(gm2, rdc_name, rdc_sqft, barcode_total_inv)
    save_data_to_csv(clean_gm, '/content/drive/MyDrive/MGT6748_Steelcase/final_clean_data_0317.csv')
    save_data_to_parquet(clean_gm, '/content/drive/MyDrive/MGT6748_Steelcase/final_clean_data_0317.parquet')



### Code to run the functions by passing the RDC's and other parameters

In [ ]:
# Loop through all the RDC's and its relevant squarefeet to generate the final data for visualization and forecating
rdc_list = ['CC', 'EP', 'GM', 'NC', 'NG', 'NT', 'WW']
rdc_sqft = [130000, 165000, 218274, 50000, 140000, 75000, 34789]

for rdc_name, rdc_sqft in zip(rdc_list, rdc_sqft):
    process_data(rdc_name, rdc_sqft)